In [1]:
import trimesh

import os
from os import path
import h5py

from src.data_gen_utils import *

import time
import sys
sys.path.append('../../../rai-fork/rai/ry')
from scipy.spatial.transform import Rotation

import libry as ry
import multiprocessing as mp

In [6]:
def inCollision(C, mugFrames):
    for name in mugFrames:
        y = -C.evalFeature(ry.FS.pairCollision_negScalar, ['hook', name])[0]
        if y < 0:
            return True
    return False

def distance(C, mugFrames):
    dist = np.inf
    for name in mugFrames:
        y = -C.evalFeature(ry.FS.pairCollision_negScalar, ['hook', name])[0]
        dist = min(dist, y)
    return dist
    

def isFeasible(C, mugFrames, size):
    mug = C.frame('mug')
    mug_pos0 = mug.getPosition()
    
    ## up
    mug_pos = mug_pos0.copy()
    cum_dist = 0.
    while True:
        dist = distance(C, mugFrames)
        if dist < 0:
            mug.setPosition(mug_pos0)
            break
        mug_pos[2] += max(abs(dist), 1e-4)
        cum_dist += max(abs(dist), 1e-4)
        if cum_dist > 2*size:
            mug.setPosition(mug_pos0)
            return False
        mug.setPosition(mug_pos)
    
    ## down
    mug_pos = mug_pos0.copy()
    cum_dist = 0.
    while True:
        dist = distance(C, mugFrames)
        if dist < 0:
            mug.setPosition(mug_pos0)
            break
        mug_pos[2] -= max(abs(dist), 1e-4)
        cum_dist += max(abs(dist), 1e-4)
        if cum_dist > 2*size:
            mug.setPosition(mug_pos0)
            return False
        mug.setPosition(mug_pos)
        
    ## left
    mug_pos = mug_pos0.copy()
    cum_dist = 0.
    while True:
        dist = distance(C, mugFrames)
        if dist < 0:
            mug.setPosition(mug_pos0)
            break
        mug_pos[1] += max(abs(dist), 1e-4)
        cum_dist += max(abs(dist), 1e-4)
        if cum_dist > 2*size:
            mug.setPosition(mug_pos0)
            return False
        mug.setPosition(mug_pos)
    
    ## right
    mug_pos = mug_pos0.copy()
    cum_dist = 0.
    while True:
        dist = distance(C, mugFrames)
        if dist < 0:
            mug.setPosition(mug_pos0)
            break
        mug_pos[1] -= max(abs(dist), 1e-4)
        cum_dist += max(abs(dist), 1e-4)
        if cum_dist > 2*size:
            mug.setPosition(mug_pos0)
            return False
        mug.setPosition(mug_pos)
    
    return True

def get_feasible_hang(N, mesh_filename, mesh_coll_filename, mass, com, size, view=False):
    feasible_poses = []
    object_trimesh = trimesh.load(mesh_filename)
    ray_intersector = trimesh.ray.ray_triangle.RayMeshIntersector(object_trimesh)
    
    C = ry.Config()
    
    hook = C.addFrame('hook').setShape(ry.ST.capsule, [.15, .002]).setColor([.4, .7, .4]).setPosition([0,0,0.8]).setRotationRad(np.pi/2, 0, 1, 0)
    hook_len = hook.info()['size'][0]
    hook_radii = hook.info()['size'][1]
    T_hook = hook.getAffineMatrix()
    
    C.addMeshFrame(mesh_coll_filename, 'mug', mass=mass, com=com).setPosition([0,0,0.5])
    mug = C.frame('mug') # mug's position can be changed because of com
    mugFrames = []
    for fname in C.getFrameNames():
        if fname[:3] == 'mug':
            C.frame(fname).setContact(1)
            mugFrames.append(fname)
            
    if view: 
        V = ry.ConfigurationViewer()
        V.setConfiguration(C)

    pos_prev, vectorZ_prev = None, None
    while True:
        # sample a collision free point & ray
        while True:
            
            if pos_prev is None:
                pos = (np.random.rand(3)-0.5)*2*(size+hook_len/2)
            else:
                pos = pos_prev + np.random.randn(3)*0.01
            
            if trimesh.proximity.signed_distance(object_trimesh, pos.reshape(1,3))[0] < -hook_radii:
                if vectorZ_prev is None:
                    vectorZ = np.random.randn(3)
                else:
                    vectorZ = pos_prev + np.random.randn(3)*0.1
                
                vectorZ /= np.linalg.norm(vectorZ)
                intersect = ray_intersector.intersects_any(ray_origins = np.stack([pos, pos], axis=0),
                                                           ray_directions = np.stack([vectorZ, -vectorZ], axis=0))
                if not intersect[0] and not intersect[1]:
                    break
                    
        vectorX = np.random.randn(3)
        vectorX -= np.dot(vectorX, vectorZ)*vectorZ
        vectorX /= np.linalg.norm(vectorX)
        rot = np.stack([vectorX, np.cross(vectorZ, vectorX), vectorZ], axis=1)
        T = np.eye(4)
        T[:3,:3] = rot
        T[:3,3] = pos-com
        
        
        T_mug = T_hook@np.linalg.inv(T)
        mug.setAffineMatrix(T_mug)
#         V.setConfiguration(C)
        if not inCollision(C, mugFrames) and isFeasible(C, mugFrames, size):
            quat = Rotation.from_matrix(rot).as_quat()[[3,0,1,2]]
            pose = np.hstack([pos, quat])
            feasible_poses.append(pose)
            pos_prev, vectorZ_prev = pos, vectorZ
            if view:
                V.setConfiguration(C)
#                 input()
                
            if len(feasible_poses) == N:
                return np.array(feasible_poses)

In [ ]:
load_dir = 'data/object'
mesh_dir = 'data/meshes'
save_dir = 'data/hang'

def start_process():
    print('Starting', mp.current_process().name)

    
def saveHangData(filename):
    print(filename)
    
    data_obj = h5py.File(path.join(load_dir, filename), mode='r')
    name = data_obj['filename'][()].decode()
    size = data_obj['size'][()]
    mass = data_obj['mass'][()]
    com = data_obj['com'][:]
    data_obj.close()
    
    mesh_filename = path.join(mesh_dir, name)  
    mesh_coll_filename = path.join(mesh_dir+'_coll', name)   
    
    feasible_poses = get_feasible_hang(1000, 
                                       mesh_filename, 
                                       mesh_coll_filename, 
                                       mass, 
                                       com, 
                                       size,
                                       view=False)
    data = h5py.File(path.join(save_dir, filename), mode='w')
    data.create_dataset("pose", data=feasible_poses, dtype=np.float32)
    data.close()
    
    
filename_list = [fn for fn in os.listdir(load_dir) if fn.endswith('.hdf5')]
# saveHangData(filename_list[0])
pool = mp.Pool(processes=31,
               initializer=start_process)
outputs = pool.map(saveHangData, filename_list)

pool.close() # no more tasks
pool.join()  # wrap up current tasks

547b3284875a6ce535deb3b0c692a2a.hdf5
